In [1]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces

/usr/local/lib/python3.11/dist-packages/gymnasium/envs/registration.py:596: UserWarning: WARN: plugin: shimmy.registration:register_gymnasium_envs raised Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gymnasium/envs/registration.py", line 594, in load_plugin_envs
    fn()
  File "/usr/local/lib/python3.11/dist-packages/shimmy/registration.py", line 304, in register_gymnasium_envs
    _register_atari_envs()
  File "/usr/local/lib/python3.11/dist-packages/shimmy/registration.py", line 205, in _register_atari_envs
    import ale_py
  File "/usr/local/lib/python3.11/dist-packages/ale_py/__init__.py", line 68, in <module>
    register_v0_v4_envs()
  File "/usr/local/lib/python3.11/dist-packages/ale_py/registration.py", line 179, in register_v0_v4_envs
    _register_rom_configs(legacy_games, obs_types, versions)
  File "/usr/local/lib/python3.11/dist-packages/ale_py/registration.py", line 64, in _register_rom_configs
    gymnasium.register(
    ^^^^^^^^^^^

In [2]:
class tictactoe(gym.Env):
    def __init__(self):
        super().__init__()
        self.observation_space = spaces.Box(low = 0, high = 2, shape = (3,3), dtype = np.int32)
        self.action_space = spaces.Discrete(9)
        self.win_reward = 1
        self.lose_reward = -1
        self.draw_reward = 0
        
        self.observation = np.zeros(shape = (3,3), dtype = np.int32)
        self.player = 1
        self.terminated = False
        self.truncated = False

        
    def reset(self):
        self.observation = np.zeros(shape = (3,3), dtype = np.int32)
        self.player = 1
        self.terminated = False
        self.truncated = False
        return self.observation

    def check(self,observation, player):
        finish = True
        for i in range(3):
            for j in range(3):
                if observation[i][j] == 0:
                    finish = False
                    break
            if not finish:
                break

        won = True
        for i in range(3):
            won = True
            for j in range(3):
                if observation[i][j] != player:
                    won = False
                    break
            if won:
                return self.win_reward, True

        for i in range(3):
            won = True
            for j in range(3):
                if observation[j][i] != player:
                    won = False
                    break
            if won:
                return self.win_reward, True

        won = True
        for i in range(3):
            if observation[i][i] != player:
                won = False
                break
        if won:
            return self.win_reward, True
        won = True
        for i in range(3):
            if observation[i][2-i] != player:
                won = False
                break
        if won:
            return self.win_reward, True

        if finish:
            return 0, True
        else:
            return 0, False
    

    def step(self,action):
        if self.observation[action//3][action%3] != 0:
            self.truncated = True
            self.terminated = True
        self.observation[action//3][action%3] = self.player

        reward, done = self.check(self.observation, self.player)
        self.player=self.player%2+1
        if done:
            self.truncated = True
            self.terminated = True
        return self.observation, reward, self.truncated, self.terminated,

In [3]:
class random_agent():
    def action(self,env):
        observation = env.observation
        possible_actions = []
        for i in range(9):
            if observation[i//3][i%3] == 0:
                possible_actions.append(i)
        return random.choice(possible_actions)




In the code below, two random agents play the game, and their score is published at the end.

In [4]:
import random
env = tictactoe()

player1 = random_agent()
player2 = random_agent()

score_player1 = 0
score_player2 = 0
for i in range(10):
    done = False
    env.reset()
    while not done:
        action = player1.action(env)
        observation, reward, trunc, term = env.step(action)
        if trunc:
            done = True
            score_player1+=reward
            print(observation, reward, 'player=1')
            break
        action = player2.action(env)
        observation, reward, trunc, term = env.step(action)
        if trunc:
            done = True
            score_player2+=reward
            print(observation, reward, 'player=2')
print('Player 1 won',score_player1, 'games. Player 2 won', score_player2)

[[0 2 2]
 [1 1 1]
 [2 1 0]] 1 player=1
[[1 2 1]
 [2 2 1]
 [2 1 1]] 1 player=1
[[2 2 2]
 [0 1 1]
 [1 1 2]] 1 player=2
[[1 0 1]
 [2 2 2]
 [1 2 1]] 1 player=2
[[1 2 1]
 [2 2 1]
 [2 1 1]] 1 player=1
[[1 1 2]
 [2 2 1]
 [1 2 1]] 0 player=1
[[2 2 1]
 [1 1 2]
 [1 1 2]] 1 player=1
[[2 0 1]
 [2 1 1]
 [2 1 2]] 1 player=2
[[0 2 1]
 [2 1 1]
 [1 2 0]] 1 player=1
[[1 0 2]
 [2 1 1]
 [0 2 1]] 1 player=1
Player 1 won 6 games. Player 2 won 3


In [5]:
class Qlearner1():
    def __init__(self):
        self.Q_table = {}

    def convert_to_tuple(self,observation):
        return tuple(tuple(row) for row in observation.tolist())
    def action(self,env):
        observation = env.observation
        observation= self.convert_to_tuple(observation)
        if observation not in self.Q_table:
            possible_actions = []
            for i in range(9):
                if observation[i//3][i%3] == 0:
                    possible_actions.append(i)
            self.Q_table[observation] = {action : 0 
                                         for action in possible_actions}
            return np.random.choice(possible_actions)
        else:
            max_Q_value = max(self.Q_table[observation].values())
            possible_actions = []
            for action in self.Q_table[observation]:
                if self.Q_table[observation][action] == max_Q_value:
                    possible_actions.append(action)
            return np.random.choice(possible_actions)
                # actions = list(self.Q_table[observation].keys())
                # probs = np.array(list(self.Q_table[observation].values()))
                # probs = np.exp(probs)
                # total = sum(probs)
                # probs = probs/total
                # return np.random.choice(actions,p = probs)
            
            

In [6]:
class Qlearner2():
    def __init__(self):
        self.Q_table = {}

    def convert_to_tuple(self,observation):
        return tuple(tuple(row) for row in observation.tolist())
    def action(self,env):
        observation = env.observation
        observation= self.convert_to_tuple(observation)
        if observation not in self.Q_table:
            possible_actions = []
            for i in range(9):
                if observation[i//3][i%3] == 0:
                    possible_actions.append(i)
            self.Q_table[observation] = {action : 0 
                                         for action in possible_actions}
            return np.random.choice(possible_actions)
        else:
            max_Q_value = max(self.Q_table[observation].values())
            possible_actions = []
            for action in self.Q_table[observation]:
                if self.Q_table[observation][action] == max_Q_value:
                    possible_actions.append(action)
            return np.random.choice(possible_actions)
            # actions = list(self.Q_table[observation].keys())
            # probs = np.array(list(self.Q_table[observation].values()))
            # probs = np.exp(probs)
            # total = sum(probs)
            # probs = probs/total
            # return np.random.choice(actions,p = probs)
                

In [7]:
lr = 0.1
epsilon = 0.1
player1 = Qlearner1()
player2 = Qlearner2()
env = tictactoe()
done = False
game_score = {0: 0 , 1 : 0, 2: 0}
for i in range(50000):
    done = False
    env.reset()
    player1_pairs = []
    player2_pairs = []
    while not done:
        coin = np.random.random()
        if coin<epsilon:
            possible_actions = []
            for action in range(9):
                if env.observation[action//3][action%3] == 0:
                    possible_actions.append(action)
            action1 = np.random.choice(possible_actions)
        else:
            action1 = player1.action(env)
        obs1 = player1.convert_to_tuple(env.observation)
        player1_pairs.append((obs1,action1))
        observation2, reward, done, _ = env.step(action1)
        if done:    
            if reward == 0:
                winner = 0
                game_score[0]+=1
            else:
                winner = 1
                game_score[1]+=1
            break
        coin = np.random.random()
        if coin<epsilon:
            possible_actions = []
            for action in range(9):
                if env.observation[action//3][action%3] == 0:
                    possible_actions.append(action)
            action2 = np.random.choice(possible_actions)
        else:
            action2 = player2.action(env)
        obs2 = player2.convert_to_tuple(env.observation)
        player2_pairs.append((obs2,action2))
        observation1, reward, done, _ = env.step(action2)
        if done:
            if reward == 0:
                winner = 0
                game_score[0]+=1
            else:
                winner = 2
                game_score[2]+=1
    if winner == 1:
        for obs, action in player1_pairs:
            if obs not in player1.Q_table:
                possible_actions = []
                for i in range(9):
                    if obs[i//3][i%3] == 0:
                        possible_actions.append(i)
                player1.Q_table[obs] = {action : 0 
                                             for action in possible_actions}
            player1.Q_table[obs][action]+= lr*(1-player1.Q_table[obs][action])
        for obs, action in player2_pairs:
            if obs not in player2.Q_table:
                possible_actions = []
                for i in range(9):
                    if obs[i//3][i%3] == 0:
                        possible_actions.append(i)
                player2.Q_table[obs] = {action : 0 
                                             for action in possible_actions}

            player2.Q_table[obs][action]+= lr*(-1-player2.Q_table[obs][action])
    if winner == 2:
        for obs, action in player1_pairs:
            if obs not in player1.Q_table:
                possible_actions = []
                for i in range(9):
                    if obs[i//3][i%3] == 0:
                        possible_actions.append(i)
                player1.Q_table[obs] = {action : 0 
                                             for action in possible_actions}

            player1.Q_table[obs][action]+= lr*(-1-player1.Q_table[obs][action])
        for obs, action in player2_pairs:
            if obs not in player2.Q_table:
                possible_actions = []
                for i in range(9):
                    if obs[i//3][i%3] == 0:
                        possible_actions.append(i)
                player2.Q_table[obs] = {action : 0 
                                             for action in possible_actions}

            player2.Q_table[obs][action]+= lr*(1-player2.Q_table[obs][action])



In [8]:
print(game_score)

{0: 29842, 1: 17239, 2: 2919}


In [9]:
import random
env = tictactoe()

random_player2 = random_agent()

score_player1 = 0
score_player2 = 0
for i in range(1000):
    done = False
    env.reset()
    while not done:
        action = player1.action(env)
        observation, reward, trunc, term = env.step(action)
        if trunc:
            done = True
            score_player1+=reward
            break
        action = random_player2.action(env)
        observation, reward, trunc, term = env.step(action)
        if trunc:
            done = True
            score_player2+=reward
print('Player 1 won',score_player1, 'games. Player 2 won', score_player2)

Player 1 won 966 games. Player 2 won 0


Our Q_learner does better than random agent, but it's not perfect. How to make it perfect? 

In [10]:
import random
env = tictactoe()

random_player1 = random_agent()

score_player1 = 0
score_player2 = 0
for i in range(1000):
    done = False
    env.reset()
    while not done:
        action = random_player1.action(env)
        observation, reward, trunc, term = env.step(action)
        if trunc:
            done = True
            score_player1+=reward
            break
        action = player2.action(env)
        observation, reward, trunc, term = env.step(action)
        if trunc:
            done = True
            score_player2+=reward
print('Player 1 won',score_player1, 'games. Player 2 won', score_player2)

Player 1 won 6 games. Player 2 won 887


Thus using the Q-learning algorithm, we were able to train a simple model to learn to play Tic-Tac-Toe. 

The Q_learner 1 learned to play as Player 1, and it learns well enough that it always beats a random player. 

The Q_learner 2 also learned to play as Player 2 well enough, and is able to almost always beat the random agent. 